# 1 - Interacting with the Fitbit API

In this section we will be using the python-fitbit and the requests modules to get data from the Fitbit API. This is not the only way to do it, for example, a simple alternative would be to use the Fitbit Web API Explorer (https://dev.fitbit.com/build/reference/web-api/explore/). The steps taken here are largely outlined in this (https://towardsdatascience.com/using-the-fitbit-web-api-with-python-f29f119621ea) Towards Data Science article.

## 1.1 Setting up 

In this section we load all necessary modules, proceed with the authorization from the Fitbit API and also define the Fitbit object (from the python-fitbit module) which is used to make some GET requests to the Fitbit API.

In [ ]:
# Import necessary modules
import gather_keys_oauth2 as Oauth2 # This is a python file you need to have in the same directory as your code so you can import it
import fitbit
import pandas as pd 
from datetime import datetime
import requests

# Enter CLIENT_ID and CLIENT_SECRET
CLIENT_ID = '23QRRC'
CLIENT_SECRET = '51922a48a2df4434cc20afaac4ee97b8'

# Date after which we have data
START_DATE = "2023-03-29"

Upon execution of the cell below, you will be redirected to another tab and will be asked to login into your Fitbit account. Upon doing that you will see a page that should say something like "Authentication Complete, you may close this tab".

In [ ]:
# Authorize user
server=Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
# Save access and refresh tokens
ACCESS_TOKEN = str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN = str(server.fitbit.client.session.token['refresh_token'])
EXPIRES_AT = str(server.fitbit.client.session.token['expires_at'])

In the previous cells we completed the authorization process for the Id and Secret that is specified in the first code block. Next, we create an instance of the Fitbit object of the python-fitbit module, which will be the base object we will use to get the data we want.

In [ ]:
# Create Fitbit object which will be used to get the data
auth2_client = fitbit.Fitbit(client_id = CLIENT_ID,
                             client_secret = CLIENT_SECRET,
                             expires_at = EXPIRES_AT,
                             oauth2 = True,
                             access_token = ACCESS_TOKEN,
                             refresh_token = REFRESH_TOKEN)

## 1.2 Get data

To get Sleep related data, we will be using the requests module. The reason for that is that for some reason in the python-fitbit module, there is an older API version variable that is hardcoded and we cannot change it (it's 1 while it should be 1.2 for Sleep data). As far as we know there is an issue about that in the module's Github page, but no implemented solution.

Therefore what we did is that we used the Fitbit Web API Explorer to get the CURL of the endpoint we want to draw data from, converted it to python using the requests module, and get the data we want.

For Activity data we used the python-fitbit module. We have used several different resources to quantify activity (steps, minutes active/sedentary).


In [ ]:
def get_sleep_data(ACCESS_TOKEN, date):
    """
    Inputs:
        - ACCESS_TOKEN <str>: Access token to be used by the API call.
        - date <str> (yyyy-mm-dd): Date for which we want to pull data.
        
    Returns the data related to sleep for <date> from the Fitbit API. 
    """
    
    # Check date value
    global START_DATE
    if datetime.strptime(date, '%Y-%m-%d') < datetime.strptime(START_DATE, '%Y-%m-%d'):
        raise ValueError("date cannot be before {}".format(START_DATE))
    
    # Make API get request
    headers = {
        'accept': 'application/json',
        'authorization': 'Bearer {}'.format(ACCESS_TOKEN),
    }

    response = requests.get('https://api.fitbit.com/1.2/user/-/sleep/date/{}.json'.format(date), 
                            headers=headers)

    return response.json()


def get_activity_data(date):
    """
    Inputs:
        - date <str> (yyyy-mm-dd): Date we want the data of.
    
    Returns activity data for the specified date. Activity is quantified in terms of the elements of the resources list
    that is defined inside the function.
    """
    
    # Check date value
    global START_DATE
    if datetime.strptime(date, '%Y-%m-%d') < datetime.strptime(START_DATE, '%Y-%m-%d'):
        raise ValueError("date cannot be before {}".format(START_DATE))
   
    # Dictionary where data returned by the API will be stored
    data = {}

    # Different kinds of resources that quantify activity
    resources = [
        "minutesSedentary",
        "minutesLightlyActive",
        "minutesFairlyActive",
        "minutesVeryActive",
        "steps"
    ]
    
    # A separate API call is made for each resource
    for resource in resources:
        resourceString = "activities/" + resource
        # detailString can be one of 1min, 5min, 15min
        if resource == "steps":
            # Thought this might make more sense, feel free to change it if you think otherwise
            detailString = "1min"
        else:
            detailString = "15min"
        
        # Use fitbit module to make the API get request
        data[resource] = auth2_client.intraday_time_series(resourceString,
                                                           date,
                                                           detail_level = detailString)
        
    return data


In [ ]:
# Print sleep data
date = "2023-04-20"
get_sleep_data(ACCESS_TOKEN, date)

In [ ]:
# Print activity data
get_activity_data(date)

### Example code on how to get and transform sleep data. Just for reference. (source: https://towardsdatascience.com/using-the-fitbit-web-api-with-python-f29f119621ea)

In [ ]:
# # Can't go before 29/3/23 since there aren't any data before that
# startTime = datetime.strptime(START_DATE, '%Y-%m-%d')
# # Should change to what we want it to be - using few days of data for testing
# endTime = pd.datetime(year = 2023, month = 3, day = 31)

# allDates = pd.date_range(start=startTime, end = endTime) # CHANGE THAT SO THAT datetime MODULE IS USED INSTEAD
# allDates

In [ ]:
# date_list = []
# df_list = []
# stages_df_list = []

# # For each date in the specified time period
# for oneDate in allDates:
#     # Format date the data of which we want to retrieve
# #     oneDate = oneDate.date().strftime("%Y-%m-%d")
#     # Get sleep data for the specified date
#     oneDayData = get_sleep_data(ACCESS_TOKEN, date)
#     test = oneDayData
#     # Get number of minutes for each stage of sleep into a df
#     stages_df = pd.DataFrame(oneDayData['summary'])
#     # Get the time series data into a df
#     timeSeriesDf = pd.DataFrame(oneDayData['sleep'][0]['minuteData'])
    
#     date_list.append(oneDate)
    
#     df_list.append(timeSeriesDf)
    
#     stages_df_list.append(stages_df)

In [ ]:
# final_df_list = []

# final_stages_df_list = []

# for date, df, stages_df in zip(date_list, df_list, stages_df_list):

#     if len(df) == 0:
#         continue
    
#     df.loc[:, 'date'] = pd.to_datetime(date)
    
#     stages_df.loc[:, 'date'] = pd.to_datetime(date)
    
#     final_df_list.append(df)
#     final_stages_df_list.append(stages_df)

# final_df = pd.concat(final_df_list, axis = 0)

# final_stages_df = pd.concat(final_stages_df_list, axis = 0)

### Trying to get SpO2 or Breathing Rate (br) data

Although this function is written exactly like the one for sleep, for some reason it still gives a 403 error (probably because there are bugs on the Fitbit side of the API).

> Try with breathing rate

In [ ]:
def get_spo2_data(ACCESS_TOKEN, date):
    """
    Inputs:
        - ACCESS_TOKEN <str>: Access token to be used by the API call.
        - date <str> (yyyy-mm-dd): Date for which we want to pull data.
        
    Returns the data related to SpO2 for <date> from the Fitbit API. 
    """
    
    # Check date value
    spo2_date = "2023-04-21"
    if datetime.strptime(date, '%Y-%m-%d') < datetime.strptime(spo2_date, '%Y-%m-%d'):
        raise ValueError("date cannot be before {} because there is no data before that date.".format(START_DATE))
    
    # Make API get request
    headers = {
        'accept': 'application/json',
        'authorization': 'Bearer {}'.format(ACCESS_TOKEN),
    }

    response = requests.get('https://api.fitbit.com/1/user/-/spo2/date/{}/all.json'.format(date), 
                            headers=headers)

    return response # response.json

date = "2023-04-21"
get_spo2_data(ACCESS_TOKEN, date)

## General example - ignore

This is just an example of how to get data in general. Can be deleted, left it here for reference.

In [ ]:
# This is the date of data that I want. 
# You will need to modify for the date you want
oneDate = pd.datetime(year = 2023, month = 3, day = 30)
# Example
oneDayData = auth2_client.intraday_time_series('activities/heart', oneDate, detail_level='1sec')

In [ ]:
oneDayData

In [ ]:
df = pd.DataFrame(oneDayData['activities-heart-intraday']['dataset'])
# Convert obj dtype to datetime object
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.strftime('%H:%M:%S')
# Make time the df index
df = df.set_index('time')

In [ ]:
df.head()

In [ ]:
start_time = '14:00:00'
end_time = '17:00:00'

df[start_time:end_time].plot()

# 2 - Import data into MongoDB

In [ ]:
# from pymongo import MongoClient
# mClient = MongoClient('localhost', 27017)